In [1]:
from dask_saturn import SaturnCluster
from dask.distributed import Client

cluster = SaturnCluster()
client = Client(cluster)
client

INFO:dask-saturn:Cluster is ready
INFO:dask-saturn:Registering default plugins
INFO:dask-saturn:Success!


Client Scheduler: tcp://d-dkark-dinesh-ml-e7cccc4d496b430cbdc6b8eb801eab0c.main-namespace:8786 Dashboard: https://d-dkark-dinesh-ml-e7cccc4d496b430cbdc6b8eb801eab0c.community.saturnenterprise.io,Cluster Workers: 3 Cores: 12 Memory: 43.31 GiB


In [5]:
!pip install -q scikit-learn optuna

In [31]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric

import optuna
from optuna.samplers import TPESampler


In [8]:
model_df = pd.read_csv("imputed_df.csv", index_col=0)

In [9]:
model_df

,primary_type,location_description,district,ward,domestic,arrest,year,month,day,weekday,hour
0,BATTERY,STREET,2.0,3.0,False,False,2005.0,1.0,1.0,5.0,1.0
1,WEAPONS VIOLATION,RESIDENCE,5.0,9.0,False,False,2005.0,1.0,1.0,5.0,1.0
2,CRIMINAL DAMAGE,CHURCH / SYNAGOGUE / PLACE OF WORSHIP,2.0,3.0,False,False,2005.0,1.0,1.0,5.0,1.0
3,THEFT,DEPARTMENT STORE,1.0,42.0,False,True,2005.0,1.0,1.0,5.0,1.0
4,THEFT,BAR OR TAVERN,18.0,42.0,False,False,2005.0,1.0,1.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
999995,OTHER OFFENSE,STREET,2.0,3.0,False,False,2008.0,12.0,31.0,2.0,12.0
999996,BATTERY,STREET,1.0,42.0,True,False,2008.0,12.0,31.0,2.0,12.0
999997,MOTOR VEHICLE THEFT,STREET,7.0,17.0,False,False,2008.0,12.0,31.0,2.0,12.0
999998,BURGLARY,RESIDENCE,6.0,17.0,False,False,2008.0,12.0,31.0,2.0,12.0


In [44]:
X = model_df.drop('arrest', axis = 1)
y = model_df.arrest.apply(lambda x: 1 if x else 0)

In [50]:
X['district'] = X['district'].astype(np.int64)
X['ward'] = X['ward'].astype(np.int64)
X['year'] = X['year'].astype(np.int64)
X['month'] = X['month'].astype(np.int64)
X['day'] = X['day'].astype(np.int64)
X['weekday'] = X['weekday'].astype(np.int64)
X['hour'] = X['hour'].astype(np.int64)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    stratify=y,
                                                    test_size=0.25,
                                                    random_state=21)

In [48]:
y_train.value_counts()

0    529989
1    220011
Name: arrest, dtype: int64

In [52]:
cat_cols = X.columns.values.tolist()

train_data = Pool(data = X_train,
                  label = y_train,
                  cat_features = cat_cols)

eval_dataset = Pool(data = X_test,
                    label = y_test,
                    cat_features = cat_cols)


In [53]:
X_train_f, y_train_f = client.scatter([X_train, y_train])

In [66]:
def fit_reg(train_idx, val_idx, train_x, train_y, cat_cols, params):
      train_x, val_x = X_train.iloc[train_idx], X_train.iloc[val_idx]
      train_y, val_y = y_train.iloc[train_idx], y_train.iloc[val_idx]

      cat_cols = X_train.columns.values.tolist()

      train_pool = Pool(data = train_x,
                  label = train_y,
                  cat_features = cat_cols)

      val_pool = Pool(data = val_x,
                    label = val_y,
                    cat_features = cat_cols)

      model = CatBoostClassifier(
          **params,
          loss_function= 'Logloss',
          eval_metric='AUC',
          task_type='GPU',
          early_stopping_rounds=50,
          random_seed=21,
          )
    
      model.fit(train_pool, 
                eval_set=val_pool,
                verbose=0)
      y_preds = model.predict_proba(val_pool)
      return eval_metric(val_pool.get_label(), y_preds[:,1], 'AUC')
      #return roc_auc_score(val_y, y_preds)
  

In [70]:
# Objective function
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 50, 500),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'max_ctr_complexity': trial.suggest_int('max_ctr_complexity', 0, 8),
        'random_strength': trial.suggest_int('random_strength', 0, 100),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
    }
    
    cat_cols = X_train.columns.values.tolist()
    
    kf = KFold(n_splits=10, random_state=21, shuffle=True)
    scores = []

    for train_idx, val_idx in kf.split(X_train):
      result = client.submit(fit_reg, train_idx, val_idx, X_train_f, y_train_f, cat_cols, params)
      score = client.gather(result)
      scores.append(score)

    return np.mean(scores)


In [71]:
%%time
# Create study
study = optuna.create_study(direction = "maximize", sampler = TPESampler(seed=21))

# Run optimization
study.optimize(objective, n_trials=20)

[I 2021-12-04 23:39:24,953] A new study created in memory with name: no-name-91c856a7-4d38-40ef-b88c-510501eb4b6a


[0.8822942820490333]
[0.8860948296898893]
[0.8837665439758037]
[0.8833926371325325]
[0.8858184499041227]
[0.8850700331292951]
[0.8867382604521217]
[0.8838457248995057]
[0.881783118059049]


[I 2021-12-04 23:39:56,618] Trial 0 finished with value: 0.884491967761954 and parameters: {'iterations': 71, 'depth': 5, 'learning_rate': 0.05259765072681896, 'l2_leaf_reg': 1.1945462492435481, 'max_ctr_complexity': 1, 'random_strength': 5, 'bagging_temperature': 0.16184063577845234}. Best is trial 0 with value: 0.884491967761954.


[0.8861157983281878]
[0.8997025552862772]
[0.9005105391985607]
[0.8999762494773915]
[0.8993866475301515]
[0.9009166195144634]
[0.9013334567596435]
[0.9025292279095771]
[0.9007857510217169]
[0.898742410585051]


[I 2021-12-04 23:45:41,459] Trial 1 finished with value: 0.9005266370004759 and parameters: {'iterations': 349, 'depth': 5, 'learning_rate': 0.03833463003091129, 'l2_leaf_reg': 1.626138591513405, 'max_ctr_complexity': 7, 'random_strength': 13, 'bagging_temperature': 0.051582055717860724}. Best is trial 1 with value: 0.9005266370004759.


[0.9013829127219252]
CPU times: user 53.3 s, sys: 14.5 s, total: 1min 7s
Wall time: 6min 16s


In [73]:
# Run optimization
study.optimize(objective, n_trials=20)

[0.9011729156564899]
[0.9018962350792717]
[0.9014146963471952]
[0.9005806076586915]
[0.9018249518327144]
[0.9024457151839516]
[0.9037405898695046]
[0.9017146452603527]
[0.8997667605758828]


[I 2021-12-04 23:56:21,865] Trial 2 finished with value: 0.9016973496699554 and parameters: {'iterations': 273, 'depth': 9, 'learning_rate': 0.057404222025053464, 'l2_leaf_reg': 9.734366128142419, 'max_ctr_complexity': 6, 'random_strength': 38, 'bagging_temperature': 0.43139322253545526}. Best is trial 2 with value: 0.9016973496699554.


[0.9024163792355009]
[0.9015933477018165]
[0.9022101008550821]
[0.9017145111822122]
[0.9009555759951295]
[0.902471915960919]
[0.9029795084828871]
[0.904093156355288]
[0.9024258757117025]
[0.9005551532272226]


[I 2021-12-05 00:02:20,727] Trial 3 finished with value: 0.9021780505362212 and parameters: {'iterations': 371, 'depth': 5, 'learning_rate': 0.07146655899479473, 'l2_leaf_reg': 9.21847572650625, 'max_ctr_complexity': 6, 'random_strength': 52, 'bagging_temperature': 0.04691838567431964}. Best is trial 3 with value: 0.9021780505362212.


[0.902781359889953]
[0.8845754536983939]
[0.8891435316632558]
[0.8868992585933315]
[0.8840251928010967]
[0.883145368754376]
[0.8882647602670274]
[0.8870291958643496]
[0.88590179822283]
[0.8804368552939432]


...

[0.8916370746393077]
[0.8986369598840844]
[0.899543427809972]
[0.8992974519708745]
[0.8985555545328219]
[0.900076368956261]
[0.9002577223193445]
[0.9019533067318684]
[0.899522974316256]
[0.8979681180681712]


[I 2021-12-05 01:29:30,666] Trial 20 finished with value: 0.8996309861695024 and parameters: {'iterations': 122, 'depth': 8, 'learning_rate': 0.0885529564249055, 'l2_leaf_reg': 9.872596990037772, 'max_ctr_complexity': 6, 'random_strength': 63, 'bagging_temperature': 0.13378459350565755}. Best is trial 11 with value: 0.9035761295689717.


[0.900497977105371]
[0.9011733873904659]
[0.9019690454941827]
[0.9015165572582776]
[0.9005758946751182]
[0.9023057509053053]
[0.9026194002248668]
[0.9037222730469179]
[0.9018609543704879]
[0.9002381616965216]


[I 2021-12-05 01:34:03,688] Trial 21 finished with value: 0.9018745897561129 and parameters: {'iterations': 211, 'depth': 6, 'learning_rate': 0.09929375041920775, 'l2_leaf_reg': 7.419679047846429, 'max_ctr_complexity': 8, 'random_strength': 73, 'bagging_temperature': 0.0681445643077473}. Best is trial 11 with value: 0.9035761295689717.


[0.9027644724989857]


In [74]:
params = study.best_trial.params
print(params)

{'iterations': 293, 'depth': 10, 'learning_rate': 0.09902880913698733, 'l2_leaf_reg': 9.863411715860835, 'max_ctr_complexity': 5, 'random_strength': 45, 'bagging_temperature': 0.044726461496793435}


In [75]:
client.shutdown()

TypeError: object NoneType can't be used in 'await' expression